In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ai-companion-py
!{sys.executable} -m pip install simplematrixbotlib
!{sys.executable} -m pip install "matrix-nio[e2e]"
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import simplematrixbotlib as botlib
import ai_companion_py as companion
import os
import urllib.request
import requests
import json

In [ ]:
AI_MODEL_LINK = "https://huggingface.co/TheBloke/Wizard-Vicuna-7B-Uncensored-GGML/resolve/main/Wizard-Vicuna-7B-Uncensored.ggmlv3.q4_K_M.bin"
model_filename = "models/Wizard-Vicuna-7B.bin"

if not os.path.exists(model_filename):
    os.makedirs("models/", exist_ok=True)
    urllib.request.urlretrieve(AI_MODEL_LINK, model_filename)
    print(f"Model downloaded and saved as {model_filename}")

In [ ]:
config = botlib.Config()
config.encryption_enabled = True
config.emoji_verify = True
config.ignore_unverified_devices = True
config.store_path = './crypto_store/'

In [ ]:
MATRIX_HOMESERVER = "https://matrix-client.matrix.org"   # change this if needed 
BOT_ACCOUNT_LOGIN = "BOT_ACCOUNT_LOGIN"                  # change this to bot account login 
BOT_ACCOUNT_PASSWORD = "BOT_ACCOUNT_PASSWORD"            # change this to bot account password 

In [ ]:
creds = botlib.Creds(MATRIX_HOMESERVER, BOT_ACCOUNT_LOGIN, BOT_ACCOUNT_PASSWORD)
bot = botlib.Bot(creds, config)

In [ ]:
BOT_NAME = "ai"
BOT_PERSONA = "ai is a artificial intelligence chatbot chatting with user"
FIRST_MESSAGE = "Hello user!"

In [ ]:
def ai_config():
    companion.init_default()
    companion.change_companion_data(BOT_NAME, BOT_PERSONA, "", FIRST_MESSAGE, 1, 3, True)

In [ ]:
@bot.listener.on_message_event
async def echo(room, message):
    match = botlib.MessageMatch(room, message, bot)
    if match.is_not_from_this_bot():
        await bot.api.send_text_message(room.room_id, companion.prompt(message.body, model_filename))

In [ ]:
if __name__ == "__main__":
    ai_config()
    companion.prompt("Hi, how are you?", model_filename)
    bot.run()